# HTML から情報を抽出する方法を学ぶ

このNotebookで、Pythonを用いてHTMLから必要な情報を抽出するための手法を学ぶことができます。

## HTML とは

**HTML(HyperText Markup Language) は、「 Web ページを記述するためのマークアップ言語」です**。最新の HTML5 は、 HTML 形式でも XML 形式でも記述できます(※1)。 HTML を定義が厳密な XML に寄せ、 Web ページのデータ化を推進する XHTML (※2)がかつて検討されていましたが、現在この流れはなくなりました。つまり、 HTML からのデータ抽出は今後も完全に機械的に行うことは困難です。

![html_history.jpg](images/html_history.jpg)

[「HTMLの方向性とXMLの位置付け～HTML5の概要と注目機能～」より引用](http://x-plus.utj.co.jp/xml-exp/32-tokushu.html)

※1: HTML5の標準化はHTML/XMLをパースした後のDOMのレベルで行われています。
※2: XHTMLはXBRLの表示に利用されています。

HTML は、パーサーと呼ばれるツールで解析をします。パーサーは HTML (もしくは XML )の文字列を、プログラムから扱えるオブジェクトのツリーに変換します。オブジェクトのツリーを Document Object Model 、DOM と呼びます。シリアライザは、逆にDOMをHTML/XMLに変換します。

![html_parser.png](images/html_parser.png)

[「Constructing the Object Model」より引用](https://web.dev/critical-rendering-path-constructing-the-object-model/)

## Python による HTML からの情報抽出

[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) は Python からパーサーを操作するための代表的なライブラリです。 BeautifulSoup からパーサーを操作することで、 HTML / XML を Python オブジェクトのツリーに変換できます。  
BeautifulSoup からは、 `html.parser` 、 `lxml` 、 `html5lib` などのパーサーが利用できます。パーサーによって実行速度やパースの方法に違いがあります(詳細は[ドキュメント](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)をご参照ください)。 Python 標準の `html.parser` 以外は、別途インストールが必要です。

なお、 BeautifulSoup は MIT ライセンスのソフトウェアです。会社で使用する場合は [Tidelift のサポート](https://tidelift.com/subscription/pkg/pypi-beautifulsoup4)を受けることもできます。

HTML からの情報抽出は基本的に次の 2 ステップです。

1. 目的の情報がある HTML 要素を取得する
2. HTML 要素からデータを抽出する

1 ができてしまえば、 2 は比較的簡単です。本 Notebook では、 1 の方法として「検索」と「移動」を学びます。

## Exercise1: 目的の HTML 要素を検索する

目的の情報があるHTML要素を取得する一番簡単な方法は、検索することです。 [`find`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) を利用することで検索ができます。

唐突ですが、 AWS Japan のオフィスは目黒にあります。山手線の目黒駅周辺の駅をいくつかピックアップし、 HTML で表現してみました。

In [1]:
html_content = """
<!DOCTYPE html>
<html>
   <head>
      <style>
         #yamanote table, td {border: 1px solid silver;}
         .aws {font-weight:bold; color:#f78e00}
      </style>
   </head>
   <body>
      <table id='yamanote' class='yamanote' border="1">
         <tr><td id='oosaki'>大崎</td></tr>
         <tr><td id='gotanda'>五反田</td></tr>
         <tr><td id='meguro' class='aws'>目黒</td></tr>
         <tr><td id='ebisu'>恵比寿</td></tr>
         <tr><td id='shibuya'>渋谷</td></tr>
      </table>
   </body>
<html>
"""

このHTMLは、表示すると次のようになります。

In [2]:
from IPython.display import HTML


HTML(html_content)

大崎
五反田
目黒
恵比寿
渋谷


これから、 BeautifulSoup を使用しこの HTML から「目黒」の HTML 要素を取得します。はじめに、 BeautifulSoup で HTML を読み込みます。

In [3]:
from bs4 import BeautifulSoup


html = BeautifulSoup(html_content.strip())

次に、目黒の HTML 要素を取得してみましょう。ヒントとして、恵比寿を取得するコードを掲載します。

In [4]:
# 恵比寿の HTML 要素の id を指定して検索する
ebisu = html.find(id="ebisu")
ebisu

<td id="ebisu">恵比寿</td>

目黒の HTML 要素を取得するコードを次のセルに実装してみてください。取得した HTML 要素は `meguro` という変数に入れてください。

In [5]:
# 目黒の HTML 要素を取得するコードを実装する

上手く取得できているか、次のセルを実行すると確認できます。

最初はエラーが表示されていますが、 `meguro` の変数に目黒の HTML 要素を入れたうえで実行すればエラーが消えるはずです。

 `.string` で取得した HTML 要素の中にあるテキストを取得しています。これが冒頭の「 2. HTML 要素からデータを抽出する」に相当します。

In [6]:
assert meguro.string == "目黒"

NameError: name 'meguro' is not defined

[`find`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) は、検索条件に一致する単一の要素を取得します。検索条件に当てはまる HTML 要素が複数ある場合は`find_all`を使います。

In [7]:
html.find_all("td")

[<td id="oosaki">大崎</td>,
 <td id="gotanda">五反田</td>,
 <td class="aws" id="meguro">目黒</td>,
 <td id="ebisu">恵比寿</td>,
 <td id="shibuya">渋谷</td>]

CSSの指定に慣れている方はCSSセレクタを使用した検索を`select`で行うことができます。

In [8]:
html.select(".aws")

[<td class="aws" id="meguro">目黒</td>]

## Exercise2: 目的の HTML 要素へ移動する

先程は確実な目印として `meguro` という `id` の属性がありましたがそれがない場合はどうすればよいでしょうか? 実際の HTML ではそうしたことが良くあります。

In [9]:
html_content_without_id = """
<!DOCTYPE html>
<html>
   <head>
      <style>
         #yamanote table, td {border: 1px solid silver;}
         .aws {font-weight:bold; color:#f78e00}
      </style>
   </head>
   <body>
      <table id="yamanote">
         <tr><td>大崎</td></tr>
         <tr><td>五反田</td></tr>
         <tr><td>目黒</td></tr>
         <tr><td>恵比寿</td></tr>
         <tr><td>渋谷</td></tr>
      </div>
   </body>
<html>
"""

この場合、近くの確実な目印まで一旦到達し、そこから移動して到達する方法が考えられます。

1. 全ての駅の HTML 要素 ( `td` ) のうち、3 番目の要素を目黒として取得する。
2. 「目黒」というてテキストを検索して、テキストが目黒である HTML 要素を取得する。

実際に行ってみましょう。まず、 BeautifulSoup で HTML を読み込みます。

In [10]:
html_without_id = BeautifulSoup(html_content_without_id.strip())

目黒の手前の五反田に到達してみます。五反田は 2 つめですが、プログラムで指定する時は 0 番目を含むので 1 を指定します。

In [11]:
gotanda = html_without_id.find_all("td")[1]
gotanda

<td>五反田</td>

`find_all` ですべての駅を検索し、 2 つめを五反田として取得しました。

次に、五反田のテキストを検索し、五反田のテキストを含む HTML 要素を取得してみましょう。

In [12]:
gotanda = html_without_id.find(text="五反田").parent
gotanda

<td>五反田</td>

`find` で五反田のテキストを検索し、 `parent` でテキストを含む(親となる) HTML 要素を取得しました。

1 と 2 、お好きな方で目黒を取得してみてください。

In [13]:
# 目黒の HTML 要素を取得するコードを実装する

五反田のセル (`<td>`) のさらに `parent` は、行 (`<tr>`) になります。 HTML のテーブルは、行 (`<tr>`) の中にセル (`<td>`) が何個かあるという形式で定義されています。

In [14]:
gotanda.parent

<tr><td>五反田</td></tr>

`parent` とは逆に、子となる要素は `children` / `contents` で取得できます。

In [15]:
html_without_id.find("table").contents

['\n',
 <tr><td>大崎</td></tr>,
 '\n',
 <tr><td>五反田</td></tr>,
 '\n',
 <tr><td>目黒</td></tr>,
 '\n',
 <tr><td>恵比寿</td></tr>,
 '\n',
 <tr><td>渋谷</td></tr>,
 '\n',
 '\n']

隣の要素へは `next_element` や `previous_element` で移動できます。五反田の行はセルが 1 つしかないので隣はない気がしますが、セルの `<td>` タグの隣にある 「五反田」 のテキストが取得されます。 

In [16]:
gotanda.next_element

'五反田'

同じ親を持つ兄弟要素を取得するには `next_sibling` 、 `previous_sibling` を使います。 五反田の行 `<tr>` の隣は目黒の行という気がしますが、テキスト要素も含むため隣の改行文字が取得されます。

In [17]:
gotanda.parent.next_sibling

'\n'

あまりきれいでない HTML の場合、 `next` や `previous` でなにが取得されるか予想は困難です。そのため、 `find_next` や `find_previous` 、 `find_next_siblings` や `find_previous_siblings` で意図した要素を指定して検索することをお勧めします。 `find_next` と `find_previous` は内部的に `next_element` / `previous_element` を使っており、`find_next_siblings` と `find_previous_siblings` は `next_siblings` / `previous_siblings` を使っています。

In [18]:
gotanda.parent.find_next("tr")

<tr><td>目黒</td></tr>